# CS492 전산학특강<인공지능 산업 및 스마트에너지>
## Deep Learning Practice 
#### Prof. Ho-Jin Choi
#### School of Computing, KAIST

---

## 6 - Appling RNNs to Other Domains
### 6-1. Text generation with an RNN 
In this practice, we will learn how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.


The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

- The model is **character-based**. When training started, the model did not know how to spell an English word, or that words were even a unit of text.
- The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.
- As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

#### Import libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import numpy as np
import os
import time

#### Download the Shakespeare dataset
Change the following line to run this code on your own data.

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)

#### Read the data
First, look in the text:

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

#### Vectorize the text
Before training, we need to **map strings to a numerical representation**. Create two lookup tables: one mapping **characters to numbers**, and another for **numbers to characters**.

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = 
idx2char = 

# 1D integer vector for all characters in the text data
text_as_int = 
print(text_as_int)

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

#### The prediction task
Given a character, or a sequence of characters, **what is the most probable next character?** This is the task we're training the model to perform. The **input to the model will be a sequence of characters**, and we train the model to **predict the output—the following character at each time step**.



#### Create training examples and targets
Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say s`eq_length` is 4 and our text is "Hello". **The input sequence would be "Hell", and the target sequence "ello"**.

To do this first use the [`tf.data.Dataset.from_tensor_slices`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#from_tensor_slices) function to convert the text vector into a stream of character indices.
- `tf.data.Dataset.from_tensor_slices`: Creates a Dataset whose elements are slices of the given tensors.

In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Make char dataset (in the form of integer)
char_dataset =

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

The `batch` method lets us easily convert these individual characters to sequences of the desired size.
- `drop_remainder`: (Optional.) A tf.bool scalar tf.Tensor, representing whether the last batch should be dropped in the case it has fewer than batch_size elements; the default behavior is not to drop the smaller batch.

In [ ]:
# Make sequences with sequence length +1
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [ ]:
def split_input_target(chunk):
    input_text = 
    target_text = 
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [ ]:
for input_example, target_example in dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

**Each index of these vectors are processed as one time step**. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the **RNN considers the previous step context in addition to the current input character**.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

#### Create training batches
We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to **shuffle the data and pack it into batches**.

[`tf.data.Dataset.shuffle`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#shuffle)(buffer_size, seed=None, reshuffle_each_iteration=None)

Randomly shuffles the elements of this dataset.

[`tf.data.Dataset.batch`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#batch)(batch_size, drop_remainder=False)

Combines consecutive elements of this dataset into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Shuffle the data and create batches (1 data = (100, 100) ==> 0:99, 1:100)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

#### Build The Model
Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

- `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
- [`tf.keras.layers.GRU`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/GRU): A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)
<img src=https://miro.medium.com/max/2400/1*dhq14CzJijlqjf7IlDB0uw.png>

 
- `tf.keras.layers.Dense`: The output layer, with vocab_size outputs.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Embedding, GRU, Dense 

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        
        # Add embedding layer
        
        # Add GRU 
        
        # Add Dense layer
    ])
    return model

In [ ]:
model = build_model(
    
)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

<img src=https://www.tensorflow.org/tutorials/text/images/text_generation_training.png>

#### Try the model
Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") 

In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [ ]:
model.summary()

To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

_Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop._

Try it for the first example in the batch:

In [ ]:
# 1D vector (100, 1)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices.shape)

This gives us, at each timestep, a prediction of the next character index:
- [`tf.squeeze`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/squeeze): Removes dimensions of size 1 from the shape of a tensor.

In [ ]:
# (100,)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

Decode these to see the text predicted by this **untrained model**:

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

#### Train the model
At this point the problem can be treated as a standard classification problem. **Given the previous RNN state, and the input this time step, predict the class of the next character.**

##### Attach an optimizer, and a loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Get loss value from untrained model
example_batch_loss  = 
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [ ]:
model.compile(

)

#### Configure checkpoints
Use a [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(

)

#### Generate text
First, restore the latest checkpoint. Then, to keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

**To run the model with a different**`batch_size`, we need to **rebuild the model and restore the weights from the checkpoint**.

In [ ]:
from google.colab import drive

drive.mount('/gdrive')

In [ ]:
import os

gdrive_root = '/gdrive/My Drive'
print('In gdrive :', os.listdir(gdrive_root))

notebook_dir = os.path.join(gdrive_root, 'Colab Notebooks')
print('In Colab Notebooks :', os.listdir(notebook_dir))

In [ ]:
checkpoint_dir = os.path.join (notebook_dir, 'cs492_day6_checkpoints')

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
# Rebuild the model by changing batch size (=1) to predict new text

# Load the weight of the model we trained 

# Change the batch size from 64 to 1


In [ ]:
model.summary()

#### The prediction loop

The following code block generates the text:

* It starts by choosing a **start string**, initializing the RNN state and setting the number of characters to generate.
* Get the **prediction distribution of the next character using the start string** and the RNN state.
* Use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.
* The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize and make paragraphs, and it imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"ROMEO: "))

The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.